## Import Libraries

In [41]:
import os
import sys

## Read text from PDF
##### Provide the path of PDF file before initialized everything

In [42]:
data='data.pdf'

In [43]:
data

'data.pdf'

## Extract data by reading

In [44]:
import PyPDF2
from docx import Document

In [45]:
def extract_text_pdf(pdf):
    text = ""
    reader = PyPDF2.PdfReader(pdf)
    for page in reader.pages:
        text += page.extract_text() or ""
    return text

In [46]:
def extract_text_docx(docx):
    doc = Document(docx)
    text = ""
    for paragraph in doc.paragraphs:
        text += paragraph.text + "\n"
    return text

In [47]:
'''def get_pdf_text(pdf_docs):
    text =""
    for pdf in pdf_docs:
        pdf_reader =PyPDF2.PdfReader(pdf_docs) 
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text'''



'''The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'''


'The file can be any pdf or any docs. read the pdf/docs and go through each and every pages and extract the text'

In [48]:
def extract(doc):
    _, file_extension = doc.split('.')
    print(file_extension)
    if file_extension == 'pdf':
        return extract_text_pdf(doc)
    elif file_extension == 'docx':
        return extract_text_docx(doc)
    else:
        raise ValueError("Unsupported file format")

In [49]:
text_data=extract(data)


pdf


In [50]:
#print(text_data)

### Convert each text into chunks
##### need to split the text using CharacterTextSplitter (which is basically present in Langchain.text_splitter) such that it should not increase token size


In [51]:
from langchain.text_splitter import CharacterTextSplitter

In [52]:
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size =1300,
    chunk_overlap =300,
    length_function =len,

    )

In [53]:
chunks = text_splitter.split_text(text_data)

#taking the particular text & splitting based on information

In [54]:
len(chunks)

61

In [55]:
print(chunks[12])

cal hrs.)  Introduction  to System  Development  environment,  The Object  paradigm,  Objec - 
tory  S/W  development  process,  Object  Oriented  Modelling  using  UML,  Use case  diagrams, 
Class  diagram,  Interaction  diagram, Class  declaration, Method  implementation, Informa - 
tion hiding and Access modifiers, Class Hierarchy (inheritance), Polymorphism, Packages 
and Data  abstraction.  Practical  using  Rational  Rose  and OO Programming  using  Java/C++ 
Evaluatio n Method:  Practical  Examination  30%  and End Semester  Examina - 
tion 70% (Student must obtain at least 35% from the theory paper and the 30% 
from the practical paper)  7  COM213 α: Operating  Systems  (15 lecture  hrs. & 22.5  practical  hrs.)   Introduc - 
tion, Process Management, CPU Scheduling, Deadlocks, Memory Management, File -System 
Implementation, I/O Systems.  Practical oriented  
Evaluation Method:  Practical Examination 30% and End  Semester Examina - 
tion 70% (Student must obtain at least 35% f

In [56]:
#print(chunks)

## Convert chunks into vectors(Text Embeddings)

In [68]:
import os
#Import 'OpenAIEmbeddings class'
from langchain.embeddings import OpenAIEmbeddings

from langchain_community.vectorstores import FAISS
#initialize the connection of my db

# Load environment variables from the .env file
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv()) 

# Retrieve the OpenAI API key from the environment variables
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")

# Ensure the API key is available
if OPENAI_API_KEY is None:
    raise ValueError("OpenAI API key not found. Please set it in the .env file.")

# Initialize OpenAIEmbeddings with your API key
embedding = OpenAIEmbeddings(openai_api_key = OPENAI_API_KEY)



In [69]:
#store in the vector db 
db = FAISS.from_texts(texts = chunks,embedding = embedding)

In [70]:
if os.path.exists("vectorstore/index.pkl"):
    # Load the existing vector store
    db = FAISS.load_local("vectorstore", embedding,allow_dangerous_deserialization=True)
else:
    # Create a new vector store from the text chunks
    db = FAISS.from_texts(texts=chunks, embedding=embedding)
    # Save the newly created vector store locally
    db.save_local("vectorstore")